# Create dataset

**Imports**

In [12]:
import glob
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import random
import time

**Generate balanced dataset of ~10,000 dives and non-dives**

In [31]:
birds = glob.glob('../Data/Acc_npy/*/')

arrs = []

t = time.time()

for bird in birds:
    
    print(f'PROCESSING BIRD: {bird}...')
    
    # Load data
    data = da.from_npy_stack(bird)
    
    # Sample dives
    dive_ix = da.where(data[:,-1] == 1)[0].compute()
    pos = data[da.random.choice(dive_ix, random.randint(1000, 1250), replace=False)]

    # Sample non-dives
    no_dive_ix = np.setdiff1d(np.arange(data.shape[0]), dive_ix)
    neg = data[da.random.choice(no_dive_ix, random.randint(1150, 1400), replace=False)]

    # Stack, shuffle, and add to list
    data_add = da.vstack((pos, neg))
    arrs.append(data_add)
    
print(f'\nTime elapsed: {time.time() - t}')
    
print('\nStacking and shuffling...')
data_full = da.vstack(arrs)  # stack
ix = np.random.choice(data_full.shape[0], data_full.shape[0], replace=False)
data_full = da.slicing.shuffle_slice(data_full, ix)  # shuffle    

# split into train/test
print('\rSplitting into train/test...')
msk = np.random.rand(len(df)) < 0.8
train = data_full[msk]
test = data_full[~msk]

# Save in npy stack
print('\rWriting to npy stack...')
da.to_npy_stack('../Data/Reduced/npy_train/', train, axis=0)
da.to_npy_stack('../Data/Reduced/npy_test/', test, axis=0)

# Convert to df and write to csv
print('\rBringing into memory...')
df_train = train.to_dask_dataframe().compute()
df_test = test.to_dask_dataframe().compute()

df_train.iloc[:, -1] = df_train.iloc[:, -1].astype(int)
df_test.iloc[:, -1] = df_test.iloc[:, -1].astype(int)

# write to csv
print('\rWriting to csv...')
df_train.to_csv('../Data/Reduced/train_reduced_dset.csv', header=False, index=False)
df_test.to_csv('../Data/Reduced/test_reduced_dset.csv', header=False, index=False)

PROCESSING BIRD: ../Data/Acc_npy/ch_gps07_gv37846_20190206_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps11_gv37849_20190206_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps10_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps12_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps16_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps03_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps13_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps08_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps09_S1/...

Time elapsed: 2311.46582698822

Stacking and shuffling...
Bringing into memory...
Writing to csv...


## With bird ID

In [26]:
birds = glob.glob('../Data/Acc_npy/*/')

dfs = []

t = time.time()

for bird in birds:
    
    print(f'PROCESSING BIRD: {bird}...')
    
    idd = bird.split('/')[-2]
    
    # Load data
    data = da.from_npy_stack(bird)
    
    # Sample dives
    dive_ix = da.where(data[:,-1] == 1)[0].compute()
    pos = data[da.random.choice(dive_ix, random.randint(1000, 1250), replace=False)]

    # Sample non-dives
    no_dive_ix = np.setdiff1d(np.arange(data.shape[0]), dive_ix)
    neg = data[da.random.choice(no_dive_ix, random.randint(1150, 1400), replace=False)]

    # Stack, compute, and add to list
    data_add = dd.from_dask_array(da.vstack((pos, neg))).compute()
    data_add.columns = [*data_add.columns[:-1], 'Dive']  # rename last col
    data_add['Dive'] = data_add['Dive'].astype(int)
    data_add['BirdID'] = idd
    
    dfs.append(data_add)
    
print(f'\nTime elapsed: {time.time() - t}')
    
print('\nStacking and shuffling...')
data_full = pd.concat(dfs, ignore_index=True) # stack
data_full = data_full.sample(frac=1) # shuffle

print('Writing to csv...')
data_full.to_csv('../Data/Reduced/reduced_dset_wnames.csv', header=True, index=False)

PROCESSING BIRD: ../Data/Acc_npy/ch_gps07_gv37846_20190206_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps11_gv37849_20190206_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps10_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps12_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps16_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps03_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps13_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps08_S1/...
PROCESSING BIRD: ../Data/Acc_npy/ch_gps09_S1/...

Time elapsed: 2292.1076719760895

Stacking and shuffling...
Writing to csv...


# From npz stack

In [37]:
import pandas as pd
import numpy as np
import random
import glob
from multiprocessing import Pool

In [31]:
file = '../Data/Acc/ch_gps03_S1/6.npz'

In [32]:
file2 = '../Data/Acc/ch_gps03_S1/0.npz'

In [40]:
files = glob.glob('../Data/Acc/ch_gps03_S1/*.npz')
files

['../Data/Acc/ch_gps03_S1/9.npz',
 '../Data/Acc/ch_gps03_S1/11.npz',
 '../Data/Acc/ch_gps03_S1/20.npz',
 '../Data/Acc/ch_gps03_S1/22.npz',
 '../Data/Acc/ch_gps03_S1/19.npz',
 '../Data/Acc/ch_gps03_S1/4.npz',
 '../Data/Acc/ch_gps03_S1/6.npz',
 '../Data/Acc/ch_gps03_S1/0.npz']

In [27]:
def process_npz(filename):
    """Loads and withdraws dive data from npz file
    """
    idd = filename.split('/')[-2]
    
    # Load
    data = np.load(filename)
    data = data[data.files[0]]
    
    # Sample all dives
    pos = data[data[:,-1] == 1, :]

    # Sample non-dives
    no_dive_ix = np.where(data[:,-1] == 0)[0]
    neg = data[np.random.choice(no_dive_ix, random.randint(200, 250), replace=False)]

    # Stack, compute, and add to list
    data_add = pd.DataFrame(np.vstack((pos, neg)))
    data_add.columns = [*data_add.columns[:-1], 'Dive']  # rename last col
    data_add['Dive'] = data_add['Dive'].astype(int)
    data_add['BirdID'] = idd
    
    return data_add
    

In [41]:
with Pool() as pool:
    dfs = list(pool.map(process_npz, files))

In [43]:
len(dfs)

8

In [ ]:
birds = glob.glob('../Data/Acc_npy/*/')  #!

dfs = []

t = time.time()

for bird in birds:
    
    print(f'PROCESSING BIRD: {bird}...')
    
    idd = bird.split('/')[-2]
    
    # Load data
    
    data = da.from_npy_stack(bird)
    
    # Sample dives
    dive_ix = da.where(data[:,-1] == 1)[0].compute()
    pos = data[da.random.choice(dive_ix, random.randint(1000, 1250), replace=False)]

    # Sample non-dives
    no_dive_ix = np.setdiff1d(np.arange(data.shape[0]), dive_ix)
    neg = data[da.random.choice(no_dive_ix, random.randint(1150, 1400), replace=False)]

    # Stack, compute, and add to list
    data_add = dd.from_dask_array(da.vstack((pos, neg))).compute()
    data_add.columns = [*data_add.columns[:-1], 'Dive']  # rename last col
    data_add['Dive'] = data_add['Dive'].astype(int)
    data_add['BirdID'] = idd
    
    dfs.append(data_add)
    
print(f'\nTime elapsed: {time.time() - t}')
    
print('\nStacking and shuffling...')
data_full = pd.concat(dfs, ignore_index=True) # stack
data_full = data_full.sample(frac=1) # shuffle

print('Writing to csv...')
data_full.to_csv('../Data/Reduced/reduced_dset_wnames.csv', header=True, index=False)